# Load needed modules

In [1]:
# modules:
import pandas as pd
import os
from openai import OpenAI


# get external variables or functions:
import src.API_key as key


from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback


import json


# Load data to translate
> this should be the drawn concept data set whereby 1 row is 1 drawn concept

In [2]:
print(os.getcwd())
# Get the current working directory
os.chdir("../01_dataPreperation/outputs") 
directory = os.getcwd()

/home/fenn/Desktop/Publications/Article_CareRobots/Analyses/02_translateData


In [3]:
# print the current working directory
print(directory)

/home/fenn/Desktop/Publications/Article_CareRobots/Analyses/01_dataPreperation/outputs


In [4]:
# List files in the current working directory
files = os.listdir('.')
# Display the list of files
print(files)

['ds.rds', 'ds.csv', 'ds.xlsx', 'ds.sav']


In [5]:
# Load the .xlsx file into a DataFrame
df = pd.read_excel(directory + "/ds.xlsx")
# Specify the columns to keep
columns_to_keep = ['CASE', 'A012_06a', 'A014_01', 'A014_02', 'A014_03', 'A111_06a', 'A112_01', 'A112_02', 'A112_03', 'A211_06a', 'A212_01', 'A212_02', 'A212_03', 'A311_06a', 'A312_01', 'A312_02', 'A312_03', 'D102_06a', 'D103_01', 'D103_02', 'D103_03', 'D202_06a', 'D203_01', 'D203_02', 'D203_03', 'D302_06a', 'D303_01', 'D303_02', 'D303_03', 'D402_06a', 'D403_01', 'D403_02', 'D403_03'] # only the df column 'text' will be kept

# Filter the DataFrame to keep only the specified columns
df = df[columns_to_keep]

# prepare the data frames 
# for the text only the unique values are kept
print("Number of rows/ colums:")
print(df.shape)

Number of rows/ colums:
(369, 33)


In [6]:
before = len(df)
df = df.loc[~df.drop(columns=['CASE']).isna().all(axis=1)].reset_index(drop=True)
after = len(df)
print(f"Removed {before-after} rows where all non-CASE columns were missing. New shape: {df.shape}")

Removed 52 rows where all non-CASE columns were missing. New shape: (317, 33)


In [7]:
df

,CASE,A012_06a,A014_01,A014_02,A014_03,A111_06a,A112_01,A112_02,A112_03,A211_06a,...,D203_02,D203_03,D302_06a,D303_01,D303_02,D303_03,D402_06a,D403_01,D403_02,D403_03
0,513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"it looks unsafe, and not nearly as gentle as i...","I would only be afraid of it hurting, or dropp...",NaN,I think it could be great in most cases. I wo...,I would worry it could be hacked,making sure patient information is secure,NaN,it could be very helpful for patients going th...,"it would need to careful, and safe",none
1,514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"Yes, there are several risks and ethical conce...","First, patient safety and dignity must always ...",NaN,I would accept the use of robots that monitor ...,"Yes, there are several risks and ethical conce...","First, patient safety and accuracy are paramou...",NaN,I would accept the use of mobility-assistance ...,One key concern is patient safety. If the robo...,Patient safety and well-being are paramount. T...
2,515,NaN,me preocupa que el robot lastime a alguna persona,me preocupa la privacidad,deberia haber un profesional siempre presente,NaN,me da desconfianza que el robot vaya a fallar ...,me preocupa que hay muchas personas que están ...,dberia de usarse siempre en presencia de un pr...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,516,NaN,"Me preocuparia la seguridad, de que solo las p...","Preocupacion etica, no, creo que no deberia ha...",Solo lo de que si se vea mucho la seguridad de...,NaN,Solo creo que tendria que usarse bajo supervis...,"No, creo que eticamente hablando es aceptable","No en el desarrollo del robot en si, mas la ed...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,522,NaN,El robot parece muy lento como para permitir h...,Que puedan aumentar los errores al no haber ve...,Riesgo de pérdida de confidencialidad para los...,NaN,Que por el momento el robot se observa algo to...,Me preocupa sobre todo que pueda llegar a last...,Asegurar la seguridad del paciente. Aun no se ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,896,NaN,"Ahorraria tiempo, mantiene un mejor orden, y e...",En hospitales de paises en desarrollo (sobre t...,Que tenga seguridad para evitar robo de insumos.,NaN,El robot puede evitar alguna lesion en el pers...,"No, el robot esta suplantando un trabajo meram...",Que sea facil de usar y que pase todas las pru...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,898,NaN,Pienso que un robot que tenga suministros y lo...,El mayor riesgo o preocupación que veo es que ...,Primero se debe asegurar que tenga protocolos ...,NaN,Pienso que este robot podría ser util pero por...,"Si, que el robot no funcione bien y lastime al...",Se debe considerar la cuestión de si es lo suf...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,906,NaN,"Me parece que puede ser de gran utilidad, incl...","Ninguno en el sentido ético, todo dependerá de...",Ninguna en particular.,NaN,"Facilitaría el traslado de los pacientes, evit...",Mientras el personal esté calificado para el m...,Ninguna.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
315,907,NaN,"Puede evitar errores humanos de desviarse, ent...","No le veo ninguna, me agrada",que no se pueda interceptar o ¨robar¨ los cont...,NaN,Conozco a muchos enfermeros que se han lesiona...,No veo alguno en este momento,"Considerar la fragilidad de los pacientes, y s...",No lo recomiendo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df["D102_06a"].unique()

array([nan, 'Nursing homes',
       'I think such a robot could be usefull in office settings as well',
       'Outpatient Surgery Centers',
       "I don't think this particular robot is that useful in any of these settings.",
       'In home homehealth care and in home hospice care as well as inpatient hospice facilities',
       'Clinics',
       'None. I think there are too many negative implications to incorporate this robot into healthcare'],
      dtype=object)

## keep only variables containing answers in Spanish

In [9]:
# Specify the columns to keep
columns_to_keep = ['CASE', 'A012_06a', 'A014_01', 'A014_02', 'A014_03', 'A111_06a', 'A112_01', 'A112_02', 'A112_03', 'A211_06a', 'A212_01', 'A212_02', 'A212_03', 'A311_06a', 'A312_01', 'A312_02', 'A312_03'] # only the df column 'text' will be kept

# Filter the DataFrame to keep only the specified columns
df = df[columns_to_keep]

# prepare the data frames 
# for the text only the unique values are kept
print("Number of rows/ colums:")
print(df.shape)

Number of rows/ colums:
(317, 17)


In [10]:
before = len(df)
df = df.loc[~df.drop(columns=['CASE']).isna().all(axis=1)].reset_index(drop=True)
after = len(df)
print(f"Removed {before-after} rows where all non-CASE columns were missing. New shape: {df.shape}")

Removed 165 rows where all non-CASE columns were missing. New shape: (152, 17)


# test dictionary

In [11]:
# Exemplary dictionary
dictionary = {
    'Hallo': 'Hello',
    'Guten Tag': 'Good day',
    'Wie sagst es dir?': 'How are you?',
    'Ich bind eine Roboter.': 'I am a robot.',
    'Ich bin ein sozialer Flofods.': 'I am a social Flofods.'
}

dictionary.update({'AAA': 'BBB',  'Hallo': 'Hello',  'aaa': '111'})

# Convert the dictionary into a DataFrame
df_test = pd.DataFrame(list(dictionary.items()), columns=['German', 'English'])

# Display the DataFrame
print(df_test)

print(dictionary.keys())

                          German                 English
0                          Hallo                   Hello
1                      Guten Tag                Good day
2              Wie sagst es dir?            How are you?
3         Ich bind eine Roboter.           I am a robot.
4  Ich bin ein sozialer Flofods.  I am a social Flofods.
5                            AAA                     BBB
6                            aaa                     111
dict_keys(['Hallo', 'Guten Tag', 'Wie sagst es dir?', 'Ich bind eine Roboter.', 'Ich bin ein sozialer Flofods.', 'AAA', 'aaa'])


# Set up ChatGPT

## Translate written text new

In [48]:
json_schema = {
    "title": "Translation",
    "description": "Dictionary that contains the words and their respective translations.",
    "type": "object",
    "properties": {
        "dictionary": {
            "type": "string",
            "description": 'Dictionary that contains the words as keys and their respective translations as values. Provide the dictionary in the following format: {"word1": "translation1", "word2": "translation2", ...}.',
        },
    },
    "required": ["dictionary"],
}

In [49]:
system_template = """<Context>You are a helpful assistant that translates {language} words to English.</Context>

<Data Structure>The lists "translate" is an array containing {language} words or sentences.</Data Structure>

<Task>Translate the provided array "translate" to English, whereby you return a dictionary with the {language} words as keys and their English translations as values. 
The dictionary should be structured according to the provided JSON schema.
Be careful with spelling errors and do not use capital letters for adjectives.
Repeat this process in case no translations are returned or the dictionary do not follow the JSON schema.</Task>
"""

user_template = """Array "translate": 
{translate}
"""

# rescue robots and socially assistive robots
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)


result = prompt_template.invoke({"language": "German", "translate":["Hallo", "Guten Tag"]})
print(result)

print("result:", result)
print("result.to_messages():", result.to_messages())

messages=[SystemMessage(content='<Context>You are a helpful assistant that translates German words to English.</Context>\n\n<Data Structure>The lists "translate" is an array containing German words or sentences.</Data Structure>\n\n<Task>Translate the provided array "translate" to English, whereby you return a dictionary with the German words as keys and their English translations as values. \nThe dictionary should be structured according to the provided JSON schema.\nBe careful with spelling errors and do not use capital letters for adjectives.\nRepeat this process in case no translations are returned or the dictionary do not follow the JSON schema.</Task>\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Array "translate": \n[\'Hallo\', \'Guten Tag\']\n', additional_kwargs={}, response_metadata={})]
result: messages=[SystemMessage(content='<Context>You are a helpful assistant that translates German words to English.</Context>\n\n<Data Structure>The lists "transla

In [70]:
def basic_API_call(
    prompt,
    language,
    translate,
    openai_api_key,
    json_schema,
    model_name="gpt-4o",
    max_tokens=1000,
    verbose=False
):

    # prompt = PromptTemplate(template=template)
    seed = 123

    model = ChatOpenAI(model=model_name, openai_api_key=openai_api_key, max_tokens=max_tokens, model_kwargs={"seed": seed}, temperature=0.0)
       
    structured_llm = model.with_structured_output(json_schema, include_raw=True)
    chain = prompt | structured_llm

    with get_openai_callback() as cb:
        response = chain.invoke(
            {"language": language, "translate": translate}
        )
        print(cb)
    
    if verbose:
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Total Cost (USD): ${cb.total_cost}")
        
    return response

In [15]:
result = basic_API_call(prompt=prompt_template,
    language="German",
    translate=["Hallo", "Guten Tag", "Wie geht es dir?"],
    openai_api_key=key.openai_api_key,
    json_schema=json_schema,
    model_name="gpt-4o",
    max_tokens=1000
)

/home/fenn/Documents/env_python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


Tokens Used: 262
	Prompt Tokens: 229
	Completion Tokens: 33
Successful Requests: 1
Total Cost (USD): $0.0009025
Total Tokens: 262
Prompt Tokens: 229
Completion Tokens: 33
Total Cost (USD): $0.0009025


In [16]:
# Extract the 'parsed' section from the JSON data
parsed_section = result.get('parsed', {})
dictionary = parsed_section.get('dictionary')
    
# Convert the string back to a Python dictionary
print(type(dictionary)) 
dictionary = eval(dictionary)
print(len(dictionary.keys()))
print(dictionary)

<class 'str'>
3
{'Hallo': 'Hello', 'Guten Tag': 'Good day', 'Wie geht es dir?': 'How are you?'}


### Apply code

In [17]:
print(os.getcwd())
os.chdir("../../02_translateData/outputs")
print(os.getcwd())

/home/fenn/Desktop/Publications/Article_CareRobots/Analyses/01_dataPreperation/outputs
/home/fenn/Desktop/Publications/Article_CareRobots/Analyses/02_translateData/outputs


In [ ]:
# if needed just test code for one variable:
df2 = df[columns_to_keep[:2]]
df2

,CASE,A012_06a
0,515,NaN
1,516,NaN
2,522,NaN
3,523,NaN
4,524,NaN
...,...,...
147,896,NaN
148,898,NaN
149,906,NaN
150,907,NaN


In [64]:
columns_to_keep

['CASE',
 'A012_06a',
 'A014_01',
 'A014_02',
 'A014_03',
 'A111_06a',
 'A112_01',
 'A112_02',
 'A112_03',
 'A211_06a',
 'A212_01',
 'A212_02',
 'A212_03',
 'A311_06a',
 'A312_01',
 'A312_02',
 'A312_03']

In [68]:
# keep the first (CASE), drop the second, and keep all other columns
df_current = df[[columns_to_keep[0]] + columns_to_keep[2:]]
df_current.columns.tolist()

['CASE',
 'A014_01',
 'A014_02',
 'A014_03',
 'A111_06a',
 'A112_01',
 'A112_02',
 'A112_03',
 'A211_06a',
 'A212_01',
 'A212_02',
 'A212_03',
 'A311_06a',
 'A312_01',
 'A312_02',
 'A312_03']

In [72]:
result

{'raw': AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_BWg5ofee2mSq6rIbtXTq8DKB', 'function': {'arguments': '{"dictionary":"{\\"Razones para aceptarlo sería la eficiencia de actividades, valoraciones generales simultáneas. Algo dudoso sería el enfoque subjetivo\\": \\"Reasons to accept it would be the efficiency of activities, simultaneous general assessments. Something doubtful would be the subjective approach.\\", \\"Un robot que lleva articulos de un lado a otro es de mucha ayuda ya que puede ser tardado cuando el hospital es grande. Esto ahorraria tiempo. Creo que si es para entregar medicamentos, muestras, etc entre el personal del hospital seria de mucha ayuda. _x000D_\\\\n_x000D_\\\\n\\": \\"A robot that carries items from one place to another is very helpful as it can be time-consuming when the hospital is large. This would save time. I think if it is for delivering medications, samples, etc. among the hospital staff, it would be very helpful.\\", \\"La te

In [ ]:
# Translate non-missing values variable-by-variable, keeping CASE numbers
chunk_size = 8
language = "Spanish"

all_translations = []

for col in [c for c in df_current.columns if c != "CASE"]:
    series = df_current[["CASE", col]].dropna(subset=[col]).reset_index(drop=True)
    if series.empty:
        print(f"Skipping {col}: no non-missing values.")
        continue

    unique_texts = series[col].astype(str).unique().tolist()

    # translate unique texts in chunks and aggregate results
    dictionary_out = {}
    counter = 0
    print(f"Translating column: {col} with {len(unique_texts)} unique texts.")
    for i in range(0, len(unique_texts), chunk_size):
        print(f"Translating chunk {counter + 1} for column {col}...")
        counter += 1
        chunk = unique_texts[i : i + chunk_size]

        result = basic_API_call(
            prompt=prompt_template,
            language=language,
            translate=chunk,
            openai_api_key=key.openai_api_key,
            json_schema=json_schema,
            model_name="gpt-4o",
            max_tokens=1000,
        )

        parsed_section = result.get("parsed", {})
        dictionary = parsed_section.get("dictionary")


        try:
            dictionary = eval(dictionary)
        except Exception as e:
            print(f"Eval error for column {col}, chunk {counter}: {e}")
            continue

        dictionary_out.update(dictionary)

    # map translations back to rows and save per-variable file
    series = series.rename(columns={col: "Original"})
    series["English"] = series["Original"].map(dictionary_out)
    series.to_excel(f"translations_{col}.xlsx", index=False)
    all_translations.append(series.assign(variable=col))

# combine all variable translations into one file
if all_translations:
    df_all = pd.concat(all_translations, ignore_index=True)[["CASE", "variable", "Original", "English"]]
    df_all.to_excel("translations_all_variables.xlsx", index=False)
    print("Saved translations_all_variables.xlsx")
else:
    print("No translations were produced.")

/home/fenn/Documents/env_python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


Tokens Used: 1182
	Prompt Tokens: 549
	Completion Tokens: 633
Successful Requests: 1
Total Cost (USD): $0.007702500000000001
Total Tokens: 1182
Prompt Tokens: 549
Completion Tokens: 633
Total Cost (USD): $0.007702500000000001


/home/fenn/Documents/env_python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


Tokens Used: 1109
	Prompt Tokens: 521
	Completion Tokens: 588
Successful Requests: 1
Total Cost (USD): $0.0071825
Total Tokens: 1109
Prompt Tokens: 521
Completion Tokens: 588
Total Cost (USD): $0.0071825


/home/fenn/Documents/env_python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

/home/fenn/Documents/env_python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


Tokens Used: 1234
	Prompt Tokens: 568
	Completion Tokens: 666
Successful Requests: 1
Total Cost (USD): $0.00808
Total Tokens: 1234
Prompt Tokens: 568
Completion Tokens: 666
Total Cost (USD): $0.00808


/home/fenn/Documents/env_python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


Tokens Used: 1782
	Prompt Tokens: 782
	Completion Tokens: 1000
Successful Requests: 1
Total Cost (USD): $0.011955
Total Tokens: 1782
Prompt Tokens: 782
Completion Tokens: 1000
Total Cost (USD): $0.011955


AttributeError: 'NoneType' object has no attribute 'get'